<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/Pandas_chunk_handle_big_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
pd.option_context("display.max_rows", 999, "display.max_columns", 999, 'max_colwidth', -1)
pd.set_option('max_colwidth', 400)

In [2]:
pd.__version__

'1.0.5'

In [3]:
# One mi rows df
n  = 1_000_000

df = pd.DataFrame(
    {
        'COL_A':[27]*n,
        'COL_B':[11]*n, 
        'COL_C':[83]*n,
     }
      )
print(df.shape)

#save
df.to_csv('bigDF.csv')

(1000000, 3)


In [39]:
class CleanData:
    def __init__(self, df):
        self.data = df.copy()

    def clean_data(self):
        data = self.data.copy()
        data = data.fillna(0)
        data = data.assign(D = ['28-01-1984']*CHUNK_SZ)
        return data

In [42]:
#read CSV in chunk
CHUNK_SZ = 100_000
df_chunk = pd.read_csv(r'bigDF.csv', index_col=0, chunksize=CHUNK_SZ)

chunk_data = [] #append each df chunk 
memory = [] #memory usage by chunk

for chunk in df_chunk:
    memory.append(chunk.memory_usage(index=True, deep=True).sum())
    
    #perform data clean
    processing = CleanData(chunk).clean_data()

    #append the data cleanead in the list
    chunk_data.append(processing)

#computes the avg memory per chunk
AVG_MEMORY = sum(memory)/len(memory)
print(f'Total number of chunks: {len(memory)} -- Avg. memory per chunk: {AVG_MEMORY*1e-6 :.2}Mb')

#concatenate the list in dataframe
df_concat = pd.concat(chunk_data) 
print(f'df concatened shape: {df_concat.shape}')

Total number of chunks: 10 -- Avg. memory per chunk: 3.2Mb
df concatened shape: (1000000, 4)
